In [1]:
import sys
sys.path.append('../')

from stepselector.data_loader import ZebraDataset, ZebraBatchSampler, custom_collate
from torch.utils.data import Dataset, DataLoader, Sampler
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.preprocessing import OneHotEncoder
import os
import joblib
from tqdm.notebook import tqdm

gdal.UseExceptions()

In [2]:
# Do you want to load five meter steps or ten meter steps?
step_length = 'five'

# Specify radius (in meters) to define social density (number of conspecifics within radius)
social_radius = 10

# How many fake steps to use per real step? (max 20)
n_ref_steps = 5

# Make list of columns to keep - these will be fetched by the dataloader
columns_to_keep = ['target_id',
                   'observation',
                   'step_speed_mps',
                   'angle_to_observers', 
                   'dist_to_observer', 
                   'delta_observer_dist', 
                   'road', 
                   'ground_class',
                   'ground_slope', 
                   'viewshed_vis', 
                   'social_dens', 
                   'social_vis',
                   'age_class',
                   'species',
                   'individual_ID']

server_mount = '/home/blair/server/herd_hover'

In [3]:
# Define data directories
data_folder = os.path.join(server_mount, 'zebra_movement_data')

# For five meter steps:
observed_steps_directory = os.path.join(data_folder, '%s_meter_steps' %step_length, 'observed')
simulated_steps_directory = os.path.join(data_folder, '%s_meter_steps' %step_length, 'simulated')


#rasters_directory = os.path.join(data_folder, 'rasters')
#ob_metadata_file = os.path.join(data_folder, 'observation_metadata.csv')
track_metadata_file = os.path.join(data_folder, 'track_metadata.csv')

In [7]:
# Initialize dataset
dataset = ZebraDataset(target_dir=observed_steps_directory, reference_dir=simulated_steps_directory, social_radius=5, num_ref_steps=5, columns_to_keep=columns_to_keep)

batch_sampler = ZebraBatchSampler(dataset)

In [8]:
# Create DataLoader
dataloader = DataLoader(dataset, batch_sampler = batch_sampler, collate_fn = custom_collate)

In [9]:
target_test = []
refs_test = []
context_test= []

# Iterate through the DataLoader
for batch in dataloader:
    target, references, context = batch
    target_test.extend(target)
    refs_test.extend(references)
    context_test.extend(context)


KeyboardInterrupt: 

In [14]:
target_test[2]

{'target_id': 'ob015_0_f714_ob',
 'observation': 'observation015',
 'step_speed_mps': -0.054746194227693064,
 'angle_to_observers': 0.2572353182827691,
 'dist_to_observer': -1.2984295410534066,
 'delta_observer_dist': 0.6853256219158823,
 'road': 0,
 'ground_slope': -1.536810753123173,
 'viewshed_vis': 0.23588930144281803,
 'social_dens': 1,
 'social_vis': 5,
 'individual_ID': '015-000',
 'ground_unclassified': 0.0,
 'ground_bare': 1.0,
 'ground_grass': 0.0,
 'ground_tree': 0.0,
 'species_gir': 0.0,
 'species_gz': 1.0,
 'species_imp': 0.0,
 'species_pz': 0.0,
 'age_adult': 1.0,
 'age_young': 0.0}

In [15]:
refs_test[2]

[{'target_id': 'ob015_0_f714_ob',
  'observation': 'observation015',
  'step_speed_mps': -0.0631384240560711,
  'angle_to_observers': 1.895706003324101,
  'dist_to_observer': -1.2872806611558985,
  'delta_observer_dist': 0.9822281321140861,
  'road': 0,
  'ground_slope': -0.9002567582807968,
  'viewshed_vis': 0.24954493213513076,
  'social_dens': 1,
  'social_vis': 5,
  'individual_ID': '015-000',
  'ground_unclassified': 0.0,
  'ground_bare': 0.0,
  'ground_grass': 1.0,
  'ground_tree': 0.0,
  'species_gir': 0.0,
  'species_gz': 1.0,
  'species_imp': 0.0,
  'species_pz': 0.0,
  'age_adult': 1.0,
  'age_young': 0.0},
 {'target_id': 'ob015_0_f714_ob',
  'observation': 'observation015',
  'step_speed_mps': -0.06993303695044,
  'angle_to_observers': -0.24055682757730493,
  'dist_to_observer': -1.321326715686033,
  'delta_observer_dist': 0.07810869022323529,
  'road': 0,
  'ground_slope': -1.651549019946122,
  'viewshed_vis': 0.1668534814607805,
  'social_dens': 1,
  'social_vis': 6,
  'in

In [16]:
context_test[2]

[{'target_id': 'ob015_0_f509_ob',
  'observation': 'observation015',
  'step_speed_mps': -1.1286230499845122,
  'angle_to_observers': -0.3184457487538329,
  'dist_to_observer': -1.4011944384099193,
  'delta_observer_dist': -0.045511074102986424,
  'road': 0,
  'ground_slope': -1.5521154295009059,
  'viewshed_vis': 0.22038390046342546,
  'social_dens': 0,
  'social_vis': 4,
  'individual_ID': '015-000',
  'ground_unclassified': 0.0,
  'ground_bare': 1.0,
  'ground_grass': 0.0,
  'ground_tree': 0.0,
  'species_gir': 0.0,
  'species_gz': 1.0,
  'species_imp': 0.0,
  'species_pz': 0.0,
  'age_adult': 1.0,
  'age_young': 0.0},
 {'target_id': 'ob015_0_f615_ob',
  'observation': 'observation015',
  'step_speed_mps': -0.1276679487924469,
  'angle_to_observers': 0.07647215781375133,
  'dist_to_observer': -1.3527832511271007,
  'delta_observer_dist': 0.5122560283783996,
  'road': 0,
  'ground_slope': -1.5529606645285827,
  'viewshed_vis': 0.271870422887441,
  'social_dens': 1,
  'social_vis': 6,